In [8]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

def get_data_b3():
    path = '../raw_data'
    files = [f for f in listdir(path) if isfile(join(path, f))] 
    list_files = []
    for file in range(0, len(files)):
        list_file = pd.read_excel(f'{path}/{files[file]}', parse_dates=['Data'])
        list_files.append(list_file)
    list_all_files = pd.concat(list_files)
    list_files_s = list_all_files.sort_values(by=['Produto', 'Data'])
    df_files = pd.DataFrame(list_files_s)
    return df_files
df_files = get_data_b3()
df_files

/home/thamiris/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação
63,Credito,2020-04-17,Transferência - Liquidação,ALSO3 - ALIANSCE SONAE SHOPPING CENTERS S.A. ...,CLEAR CORRETORA - GRUPO XP,10,24,240
62,Credito,2020-04-20,Transferência - Liquidação,ALSO3 - ALIANSCE SONAE SHOPPING CENTERS S.A. ...,CLEAR CORRETORA - GRUPO XP,19,24.5,465.5
47,Debito,2020-05-19,Transferência - Liquidação,ALSO3 - ALIANSCE SONAE SHOPPING CENTERS S.A. ...,CLEAR CORRETORA - GRUPO XP,34,20.82,707.88
85,Credito,2020-06-01,Transferência - Liquidação,ALSO3 - ALIANSCE SONAE SHOPPING CENTERS S.A. ...,CLEAR CORRETORA - GRUPO XP,5,50.5,252.5
38,Debito,2020-06-25,Transferência - Liquidação,ALUP11 - ALUPAR INVESTIMENTO S/A ...,CLEAR CORRETORA - GRUPO XP,35,24.45,855.75
...,...,...,...,...,...,...,...,...
109,Credito,2020-10-12,Dividendo,YDUQ3 - YDUQS PARTICIPACOES S.A,CLEAR CORRETORA - GRUPO XP,30,0.51,15.29
45,Credito,2021-07-12,Dividendo,YDUQ3 - YDUQS PARTICIPACOES S.A,CLEAR CORRETORA - GRUPO XP,30,0.47,14.09
16,Debito,2021-09-22,Transferência - Liquidação,YDUQ3 - YDUQS PARTICIPACOES S.A,CLEAR CORRETORA - GRUPO XP,30,23.64,709.2
66,Credito,2020-02-04,Transferência - Liquidação,YDUQ3 - ESTACIO PARTICIPACOES S.A. ...,CLEAR CORRETORA - GRUPO XP,30,23,690


In [3]:
def files_featuring_eng():
    df_files = get_data_b3()
    df_files['Data da compra'] = df_files['Data'].where(df_files['Entrada/Saída'] == 'Credito')
    df_files['Data da venda'] = df_files['Data'].where(df_files['Entrada/Saída'] == 'Debito')
    df_files = df_files[['Produto', 'Data da compra', 'Data da venda', 'Quantidade', 'Preço unitário', 'Valor da Operação', 'Movimentação', 'Instituição']]
    df_files['Quantidade'] = df_files['Quantidade'].apply(lambda x: x.replace(',','.'))
    df_files['Quantidade'] = df_files['Quantidade'].astype(float)
    return df_files
df_files = files_featuring_eng()
df_files

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [3]:
def fixed_income():
    df_files = files_featuring_eng()
    df_fixedi_b = df_files[df_files['Movimentação'] == 'Compra']
    df_fixedi_s = df_files[df_files['Movimentação'] == 'Venda']
    df_fixedi = pd.concat([df_fixedi_b, df_fixedi_s])
    df_fixedi = df_fixedi[['Produto', 'Data da compra', 'Data da venda', 'Quantidade', 'Preço unitário', 'Valor da Operação', 'Instituição']]
    df_fixedi = df_fixedi.sort_values(by=['Produto']).reset_index()
    return df_fixedi
df_fixedi = fixed_income()
df_fixedi

/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,index,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação,Instituição
0,63,Tesouro Selic 2023,NaT,2020-03-17,0.71,10551.64,7491.66,XP INVESTIMENTOS CCTVM S/A
1,78,Tesouro Selic 2025,2020-07-01,NaT,2.14,10466.52,22398.35,CLEAR CORRETORA - GRUPO XP
2,0,Tesouro Selic 2025,2022-03-23,NaT,4.72,11457.89,54081.24,CLEAR CORRETORA - GRUPO XP
3,77,Tesouro Selic 2025,NaT,2020-03-02,2.14,10495.41,22460.18,CLEAR CORRETORA - GRUPO XP
4,6,Tesouro Selic 2027,2021-10-19,NaT,7.34,10905.72,80047.98,CLEAR CORRETORA - GRUPO XP


In [4]:
def fixed_income_quantity():
    df_fixedi = fixed_income()
    df_fixedi_buy = df_fixedi[df_fixedi['Data da compra'] > '20000101'].groupby('Produto').agg({'Quantidade':'sum'}).reset_index()
    df_fixedi_sel = df_fixedi[df_fixedi['Data da venda'] > '20000101'].groupby('Produto').agg({'Quantidade':'sum'}).transform(lambda x: x *-1).reset_index()
    quantity = pd.concat([df_fixedi_sel, df_fixedi_buy]).reset_index()
    quantity_prod = quantity.groupby(['Produto']).sum().reset_index()
    quantity_prod.drop(columns='index', inplace=True)
    for line in range(0, len(quantity_prod)):
        if quantity_prod.Quantidade[line] < 0:
            quantity_prod.Quantidade[line] = quantity_prod.Quantidade[line]*-1
        else:
            quantity_prod.Quantidade[line]
    quantity_prod = quantity_prod.rename(columns={'Quantidade': 'Saldo no Tesouro'})
    quantity_prod = pd.DataFrame(quantity_prod)
  #  quantity_prod = quantity_prod.drop(columns='index')
    return quantity_prod
quantity_prod = fixed_income_quantity()
quantity_prod

/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/tmp/ipykernel_2304/569478855.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quantity_prod.Quantidade[line] = quantity_prod.Quantidade[line]*-1


,Produto,Saldo no Tesouro
0,Tesouro Selic 2023,0.71
1,Tesouro Selic 2025,4.72
2,Tesouro Selic 2027,7.34


In [5]:
def fixed_income_prof_loss():
    df_fixedi = fixed_income()
    df_fixedi_opB = df_fixedi[df_fixedi['Data da compra'] > '20000101'].reset_index()
    df_fixedi_opS = df_fixedi[df_fixedi['Data da venda'] > '20000101'].reset_index()

    df_fixedi_opB['Lucro/Prejuízo'] = 0
    for lineS in range(0, len(df_fixedi_opS)):
        for lineB in range(0, len(df_fixedi_opB)):
            if df_fixedi_opS['Quantidade'][lineS] == df_fixedi_opB['Quantidade'][lineB]:
                df_fixedi_opB['Lucro/Prejuízo'][lineB] = df_fixedi_opS['Valor da Operação'][lineS] - df_fixedi_opB['Valor da Operação'][lineB]
    prof_loss_prod = df_fixedi_opB[['Produto', 'Lucro/Prejuízo']]
    return prof_loss_prod
prof_loss_prod = fixed_income_prof_loss()
prof_loss_prod

/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/tmp/ipykernel_2304/1990415420.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fixedi_opB['Lucro/Prejuízo'][lineB] = df_fixedi_opS['Valor da Operação'][lineS] - df_fixedi_opB['Valor da Operação'][lineB]


,Produto,Lucro/Prejuízo
0,Tesouro Selic 2025,61
1,Tesouro Selic 2025,0
2,Tesouro Selic 2027,0


In [6]:
def fixed_income_b3():
    prof_loss_prod = fixed_income_prof_loss()
    quantity_prod = fixed_income_quantity()
    df_fixedi = fixed_income()
    quan_prof_loss = pd.concat([quantity_prod, prof_loss_prod]).reset_index()
    prof_loss = quan_prof_loss.groupby(['Produto']).sum().reset_index()
    prof_loss = prof_loss.drop(columns='index')

    merge_fixo = pd.merge(df_fixedi, prof_loss, how='left', on='Produto')
    merge_fixo.drop(columns=['index', 'Instituição'], inplace = True)
    merge_fixo = merge_fixo.sort_values(['Produto', 'Data da compra', 'Data da venda'])

    merge_fixo['Lucro/Prejuizo'] = ''
    merge_fixo['Saldo Tesouro'] = ''
    for line in range(0, len(merge_fixo)-1):
        if merge_fixo['Produto'].iloc[line] != merge_fixo['Produto'].iloc[line+1]:
            merge_fixo['Lucro/Prejuizo'].iloc[line] = merge_fixo['Lucro/Prejuízo'].iloc[line]
            merge_fixo['Saldo Tesouro'].iloc[line] = merge_fixo['Saldo no Tesouro'].iloc[line]
        else:
            merge_fixo['Lucro/Prejuizo'].iloc[line] = 'nan'
            merge_fixo['Saldo Tesouro'].iloc[line] = 'nan'
    merge_fixo['Lucro/Prejuizo'].iloc[len(merge_fixo)-1] = merge_fixo['Lucro/Prejuízo'].iloc[len(merge_fixo)-1]
    merge_fixo['Saldo Tesouro'].iloc[len(merge_fixo)-1] = merge_fixo['Saldo no Tesouro'].iloc[len(merge_fixo)-1]
    merge_fixo.drop(columns=['Saldo no Tesouro', 'Lucro/Prejuízo'], inplace=True)
    spreadsheet_to_income_tax = merge_fixo.to_excel('../raw_data/files_to_ti/spreadsheet_income_tax2.xlsx', index = False)
    return spreadsheet_to_income_tax
planilha = fixed_income_b3()
planilha

/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/tmp/ipykernel_2304/1990415420.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fixedi_opB['Lucro/Prejuízo'][lineB] = df_fixedi_opS['Valor da Operação'][lineS] - df_fixedi_opB['Valor da Operação'][lineB]
/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [7]:
print(planilha)

None
